# Classification d’images ASL avec un CNN
Ce projet consiste à créer et entraîner un **réseau de neurones convolutionnel** pour classifier des images de la langue des signes américaine (ASL).  
**Objectifs :**
- Préparer les données pour un modèle CNN.
- Concevoir un CNN performant avec plusieurs types de couches.
- Entraîner et évaluer le modèle sur un jeu de données d’images ASL.


1. Import des bibliothèques et configuration GPU

In [2]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

# Détection du device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


2. Chargement et préparation des données

In [3]:
# Chargement des datasets
train_df = pd.read_csv("/content/drive/MyDrive/sign_mnist_train.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/sign_mnist_valid.csv")


# Paramètres images
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

2.1 Visualiser un échantillon

In [4]:
sample_df = train_df.head().copy()
sample_df.pop('label')
sample_x = sample_df.values
sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
print("Shape des images :", sample_x.shape)

Shape des images : (5, 1, 28, 28)


3. Création du Dataset personnalisé

In [5]:
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()
        y_df = x_df.pop('label')
        x_df = x_df.values / 255 # Normalisation
        x_df = x_df.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

4. Création des DataLoaders

In [6]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

# Création des datasets
train_data = MyDataset(train_df)
valid_data = MyDataset(valid_df)

# Création des DataLoaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)  # shuffle=True pour mélanger les données
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)  # validation n'a pas besoin de shuffle

# Vérification d'un batch
batch = next(iter(train_loader))
images, labels = batch

print("Shape du batch d'images :", images.shape)  # (batch_size, channels, height, width)
print("Shape du batch de labels :", labels.shape)  # (batch_size,)

# Nombre total d'exemples
train_N = len(train_loader.dataset)
valid_N = len(valid_loader.dataset)

Shape du batch d'images : torch.Size([32, 1, 28, 28])
Shape du batch de labels : torch.Size([32])


In [7]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [8]:
batch[1].shape

torch.Size([32])

5. Création du modèle CNN

In [9]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 14 x 14
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 7 x 7
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
)
model = torch.compile(model.to(device))
print(model)

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout

6. Définition de la loss et de l’optimiseur

In [10]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

7. Fonction d’évaluation de précision

In [11]:
def get_batch_accuracy(output, y, N):
  pred = output.argmax(dim=1, keepdim=True)
  correct = pred.eq(y.view_as(pred)).sum().item()
  return correct / N

8. Fonctions d’entraînement et de validation

In [12]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()  # met le modèle en mode évaluation
    with torch.no_grad():  # pas de calcul du gradient
        for x, y in valid_loader:
            output = model(x)
            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)

    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))


In [13]:
def train():
    loss = 0
    accuracy = 0
    model.train()  # mode entraînement
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)

    # Print à l'intérieur de la fonction
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))


9. Entraînement du modèle

In [14]:
epochs = 20
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train()
    validate()


Epoch 1/20


/usr/local/lib/python3.12/dist-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
W1121 19:54:50.075000 819 torch/_inductor/utils.py:1558] [0/0] Not enough SMs to use max_autotune_gemm mode


Train - Loss: 242.0406 Accuracy: 0.9186
Valid - Loss: 31.3478 Accuracy: 0.9451
Epoch 2/20
Train - Loss: 16.9846 Accuracy: 0.9946
Valid - Loss: 41.4157 Accuracy: 0.9406
Epoch 3/20
Train - Loss: 4.3831 Accuracy: 0.9987
Valid - Loss: 12.6675 Accuracy: 0.9826
Epoch 4/20
Train - Loss: 15.5492 Accuracy: 0.9947
Valid - Loss: 31.7860 Accuracy: 0.9601
Epoch 5/20
Train - Loss: 6.1483 Accuracy: 0.9981
Valid - Loss: 32.2951 Accuracy: 0.9578
Epoch 6/20
Train - Loss: 10.8619 Accuracy: 0.9958
Valid - Loss: 25.4081 Accuracy: 0.9591
Epoch 7/20
Train - Loss: 7.9792 Accuracy: 0.9974
Valid - Loss: 44.0583 Accuracy: 0.9347
Epoch 8/20
Train - Loss: 5.9468 Accuracy: 0.9980
Valid - Loss: 33.5864 Accuracy: 0.9590
Epoch 9/20
Train - Loss: 4.8300 Accuracy: 0.9983
Valid - Loss: 23.1903 Accuracy: 0.9695
Epoch 10/20
Train - Loss: 6.0722 Accuracy: 0.9981
Valid - Loss: 15.6423 Accuracy: 0.9855
Epoch 11/20
Train - Loss: 5.9394 Accuracy: 0.9981
Valid - Loss: 30.4999 Accuracy: 0.9625
Epoch 12/20
Train - Loss: 4.3492 Acc

# 10. Résultats

Le modèle CNN atteint une bonne précision sur l’ensemble d’entraînement et l’ensemble de validation. Cette approche permet de mieux généraliser que les modèles plus simples et constitue une base solide pour d’autres projets de classification d’images.
